<a href="https://colab.research.google.com/github/leochartrand/IFT615/blob/main/CNN/CNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Cette démonstration est inspirée du cours "CS50’s Introduction to Artificial Intelligence with Python" donné à l'Université d'Harvard.*

https://cs50.harvard.edu/ai/2024/

https://creativecommons.org/licenses/by-nc-sa/4.0/





### Ce notebook a pour but d'introduire les réseaux de neurones à convolution et leur développement en Python. Nous utiliserons la librairie PyTorch, qui est aujourd'hui la plus utilisée dans le développement et la recherche en apprentissage profond. Nous créerons un réseau à convolution simple afin de classifier des images de chiffres écrits à la main.

Comme toujours, on commence par importer les librairies:

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
from tqdm import tqdm

On peut ensuite charger le jeux de données MNIST. Il s'agit d'un ensemble de 60000 images de chiffres écrits à la main. Les images sont en "greyscale" et d'une taille de 28x28 pixels. Le jeux de données est divisé en un ensemble d'entraînement et un ensemble de test. La raison pour ceci est que lors de l'évaluation, le modèle sera testé avec des images qu'il n'aura jamais vu durant l'entraînement, afin de vérifier ses capacités de "généralisation".

Notez qu'on définit une variable "batch size". En IA, une batch est un ensembe de données sur lequel le modèle va s'entraîner en même temps. Leur usage permet théoriquement un apprentissage plus stable. En pratique, elle fait aussi utilisation des capacités de parallélisme des GPUs pour un entraînement plus rapide.

In [ ]:
batch_size = 100

train_data = torchvision.datasets.MNIST(root = './', train = True,
                                        transform = torchvision.transforms.ToTensor(),
                                        download = True)

train_dataloader = torch.utils.data.DataLoader(dataset = train_data,
                                          batch_size = batch_size,
                                          shuffle = True)

test_data = torchvision.datasets.MNIST(root = './', train = False,
                                       transform = torchvision.transforms.ToTensor())

test_dataloader = torch.utils.data.DataLoader(dataset = test_data,
                                          batch_size = batch_size,
                                          shuffle = False)

Profitons-en pour observer quelques exemples du jeux de données:

In [ ]:
index, (ex_data, ex_labels) = next(enumerate(test_dataloader))

fig = plt.figure()
for i in range(9):
  plt.subplot(3,3,i+1)
  plt.axis('off')
  plt.tight_layout()
  plt.imshow(ex_data[i][0], cmap='gray')
  plt.title("Vérité terrain: %d"%(ex_labels[i]))
fig.show()

On peut maintenant instancier notre modèles et définir ses paramètres! Voici ci-dessous un exemple de réseau de neurones qui utilise une couche convolutive suivie d'une couche de pooling. Les couches convolutives sont suivies d'une couche dense cachée ainsi d'une couche dense finale qui retourne les prédictions.

Pour déterminer la taille d'entrée de la couche dense cachée, il faut comprendre comment calculer la taille de sortie d'une couche convolutive. Soit la taille des données $D$ et la taille du filtre $F$, on a:
$$D-F+1$$

En pratique, les réseaux à convolution sont beaucoup plus complexes et l'équation énoncée ci-haut comporte plus de paramètres. Cependant, dans cette exemple, seuls $D$ et $F$ suffisent à déterminer les dimensions des données en sortie.

In [12]:
model = nn.Sequential(

    # Couche convolutive. Apprend 32 filtres en utilisant un noyau 3x3
    # Arguments: 
    #   in_channels     = 1, étant donné que l'image est en greyscale. 
    #   out_channels    = 32, soit le nombre de filtres que nous voulons appliquer. 
    #   kernel_szie     = (3,3), les dimensions des filtres. 
    nn.Conv2d(1, 32, (3, 3)),   
    # Dimensions des données de sortie: (28-3+1) = 26 -> 32 filtres en 26x26

    # Activation ReLU
    nn.ReLU(),

    # Couche de max-pooling de taille 2x2
    nn.MaxPool2d((2, 2)),       
    # Dimensions des données de sortie: 26/2 = 13 -> 32 filtres en 13x13

    # On applatie les données en un seul vecteur
    nn.Flatten(),               
    # Dimensions des données de sortie: 32x13x13 = 5408

    # Couche dense
    nn.Linear(5408, 128),
    nn.ReLU(),

    # Couche dense finale avec comme sortie les probabilitées pour les 10 chiffres
    nn.Linear(128, 10),
)

Par principe de simplicité et de rapidité d'exécution, une seule couche convolutive a été introduite dans le réseau. Cependant, il normal d'en avoir plusieurs une à la suite de l'autre, possiblement en alternance avec des couches de pooling. À l'opposé des premières couches convolutives qui font ressortir des caractéristiques de bas niveau dans une image, les couches convolutives en fin de réseau permettent d'extraire des patrons de caractéristiques plus généraux et abstraits.

Le code suivant utilise la librairire PyTorch pour établir une fonction de perte ainsi qu'un algorithme d'optimisation pour la descente de gradient. Ces éléments sont tous deux responsables de l'apprentissage, tout comme avec les réseaux de neurones à couches denses. Nous ne leur porterons pas attention dans cette présentation.

In [ ]:
# On instancie un optimiseur qui s'occupe de la descente de gradient
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# On instancie notre fonction de parte
criterion = nn.CrossEntropyLoss()

# Entraînons maintenant notre modèle!

In [ ]:
# On garde un historique de l'entraînement pour visualiser nos résultats
loss_history = []

epochs = 5

for epoch in range(epochs):
  print("Epoch %d/%d"%(epoch+1, epochs))
  progress = tqdm(train_dataloader)
  for index, (data, labels) in enumerate(progress):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    loss_history.append(loss.item())
    progress.set_description("Loss: %.4f"%loss.item())

On peut visualiser l'apprentissage de notre modèle à l'aide de la librairie ***matplotlib***:

In [ ]:
plt.plot(range(epochs*len(train_dataloader)), loss_history)
plt.xlabel('Batches')
plt.ylabel('Loss')
plt.title('Loss history')
plt.show()

Évaluons maintenant la précision de notre modèle sur le jeu de données de test:

In [ ]:
correct = 0
total = 0
for data, labels in test_dataloader:
  output = model(data)
  _, predictions = torch.max(output,1)
  correct += (predictions == labels).sum()
  total += labels.size(0)

print('Précision du modèle: %.3f %%' %(100*correct/total))

On peut voir qu'un réseau à convolution assez simple peut offrir une performance remarquable. Si vous voulez, vous pouvez essayer d'améliorer l'architecture du réseau pour obtenir un meilleur score!